In [55]:
import re
from dataclasses import dataclass
import trafilatura


@dataclass
class Question:
    id: int
    topic: str
    question: str
    answer_a: str
    answer_b: str
    answer_c: str
    answer_d: str

def parse_single_question(block: str, topic: str) -> Question:
    # Find the question id
    m = re.match(r"\s*(\d+)\.\s*", block)
    if not m:
        raise ValueError(f"Invalid question block (no id):\n{block}")
    qid = int(m.group(1))
    rest = block[m.end():]

    # Split rest at the first line starting with '- '
    lines = rest.splitlines()
    question_lines = []
    answer_lines = []
    in_answers = False
    for line in lines:
        if line.strip().startswith('- '):
            in_answers = True
            answer_lines.append(line.strip())
        elif in_answers and line.strip() == '':
            continue
        elif in_answers:
            answer_lines[-1] += ' ' + line.strip()
        else:
            question_lines.append(line.strip())

    # Join the question lines and fix any artifacts
    question = ' '.join(q for q in question_lines if q)
    question = re.sub(r'\s+', ' ', question).strip()
    # Parse first four answers
    ans_list = []
    for ans in answer_lines:
        if ans.startswith('- '):
            ans_list.append(ans[2:].strip())
        if len(ans_list) == 4:
            break
    if len(ans_list) != 4:
        raise ValueError(f"Not 4 answers in:\n{block}")
    return Question(
        id=qid,
        topic=topic,
        question=question,
        answer_a=ans_list[0],
        answer_b=ans_list[1],
        answer_c=ans_list[2],
        answer_d=ans_list[3]
    )


def get_blocks(markdown: str) -> list:
    match = re.search(r'^\d+\.', markdown, re.MULTILINE)
    if not match:
        return []
    start = match.start()
    questions_text = markdown[start:]
    blocks = re.split(r'(?=^\d+\.\s)', questions_text, flags=re.MULTILINE)
    blocks = [b.strip() for b in blocks if b.strip()]
    return blocks

def parse_questions(markdown: str, topic: str) -> list:
    blocks = get_blocks(markdown)
    return [parse_single_question(block, topic) for block in blocks]

def read_elwis(url: str, topic: str) -> list:
    downloaded = trafilatura.fetch_url(url)
    markdown = trafilatura.extract(downloaded, output_format='markdown', include_tables=True)
    questions = parse_questions(markdown, topic)
    return questions


# Binnen

In [56]:
url = 'https://www.elwis.de/DE/Sportschifffahrt/Sportbootfuehrerscheine/Fragenkatalog-Binnen/Spezifische-Fragen-Segeln/Spezifische-Fragen-Segeln-node.html'
questions_binnen_segeln = read_elwis(url, topic="Binnen Segeln")

In [57]:
url = "https://www.elwis.de/DE/Sportschifffahrt/Sportbootfuehrerscheine/Fragenkatalog-Binnen/Basisfragen/Basisfragen-node.html"
questions_binnen_basis = read_elwis(url, topic="Binnen Basisfragen")

In [59]:
url = "https://www.elwis.de/DE/Sportschifffahrt/Sportbootfuehrerscheine/Fragenkatalog-Binnen/Spezifische-Fragen-Binnen/Spezifische-Fragen-Binnen-node.html"
questions_binnen_spezifisch = read_elwis(url, topic="Binnen Spezifische Fragen")

# See

In [43]:
url = 'https://www.elwis.de/DE/Sportschifffahrt/Sportbootfuehrerscheine/Fragenkatalog-See/Basisfragen/Basisfragen-node.html'
questions_see_basis = read_elwis(url, "See Basisfragen")

In [60]:
url = 'https://www.elwis.de/DE/Sportschifffahrt/Sportbootfuehrerscheine/Fragenkatalog-See/Spezifische-Fragen-See/Spezifische-Fragen-See-node.html'
questions_see_spezifisch = read_elwis(url, "See Spezifische Fragen")

In [61]:
questions_see_spezifisch[0]

Question(id=73, topic='See Spezifische Fragen', question='Wo gelten die Kollisionsverhütungsregeln (KVR)?', answer_a='Auf der Hohen See und auf den mit dieser zusammenhängenden, von Seeschiffen befahrbaren Gewässern.', answer_b='Auf der Hohen See und den deutschen Seeschifffahrtsstraßen, die von Seeschiffen befahren werden.', answer_c='Auf den Seeschifffahrtsstraßen und den küstennahen deutschen Seegewässern, die von Seeschiffen befahren werden.', answer_d='Auf der Hohen See und den von Seeschiffen befahrbaren Randmeeren, mit Ausnahme der Verkehrstrennungsgebiete.')

In [62]:
questions_see_spezifisch[-1]

Question(id=285, topic='See Spezifische Fragen', question='Was bedeutet auf einem Schiff eines der folgenden Signale?', answer_a='Fahrzeug in Seenot.', answer_b='Fahrzeug ist manövrierbehindert.', answer_c='Fahrzeug mit gefährlichen Gütern.', answer_d='Fahrzeug vor Anker mit mehr als 100 m Länge. Stand: 01. August 2023')

# Speech syntehsis

In [64]:
def question_to_text(question: Question) -> str:
    txt = f"""
Frage {question.id} - {question.topic}.

{question.question}

Antwort A: {question.answer_a}

Antwort B: {question.answer_b}

Antwort C: {question.answer_c}

Antwort D: {question.answer_d}

Die richtige Antwort ist Antwort A: {question.answer_a}
    """
    return txt

txt = question_to_text(questions_see_spezifisch[-1])
print(txt)


Frage 285 - See Spezifische Fragen.

Was bedeutet auf einem Schiff eines der folgenden Signale?

Antwort A: Fahrzeug in Seenot.

Antwort B: Fahrzeug ist manövrierbehindert.

Antwort C: Fahrzeug mit gefährlichen Gütern.

Antwort D: Fahrzeug vor Anker mit mehr als 100 m Länge. Stand: 01. August 2023

Die richtige Antwort ist Antwort A: Fahrzeug in Seenot.
    


In [65]:
JSON_TEMPLATE = f"""
{
  "audioConfig": {
    "audioEncoding": "LINEAR16",
    "effectsProfileId": [
      "headphone-class-device"
    ],
    "pitch": 0,
    "speakingRate": 1
  },
  "input": {
    "text": "{txt}"
  },
  "voice": {
    "languageCode": "de-DE",
    "name": "de-DE-Chirp3-HD-Enceladus"
  }
}
"""

ValueError: Invalid format specifier ' "LINEAR16",
    "effectsProfileId": [
      "headphone-class-device"
    ],
    "pitch": 0,
    "speakingRate": 1
  ' for object of type 'str'